In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from pylab import rcParams
import statsmodels.api as sm
import warnings
import itertools
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Bidirectional
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
prediction_data=pd.read_excel('/content/예측용_데이터.xlsx')
df=pd.read_excel('/content/학습용_데이터.xlsx')
df.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['평균기온(℃)','평균최고기온(℃)','최고기온(℃)','평균최저기온(℃)','최저기온(℃)','강수량','인구','아황산가스','일산화탄소','이산화질소','철새 합계'] ##수정 필요
df_scaled = scaler.fit_transform(df[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

df_scaled

In [ ]:
TEST_SIZE = 60

train = df_scaled[:-TEST_SIZE]
test = df_scaled[-TEST_SIZE:]

In [ ]:
from sklearn.model_selection import train_test_split

feature_cols = ['평균기온(℃)','평균최고기온(℃)','최고기온(℃)','평균최저기온(℃)','최저기온(℃)','강수량','인구','아황산가스','일산화탄소','이산화질소','철새 합계'] 
label_cols = ['철새 합계']

train_feature = train.iloc[:, :-1]
train_label = train.iloc[:, -1]

test_feature = test.iloc[:, :-1]
test_label = test.iloc[:, -1]


x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

x_train.shape, x_valid.shape

In [ ]:
prediction_data

prediction_data.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['평균기온(℃)','평균최고기온(℃)','최고기온(℃)','평균최저기온(℃)','최저기온(℃)','강수량','인구','아황산가스','일산화탄소','이산화질소','철새 합계'] ##수정 필요
pdf_scaled = scaler.fit_transform(prediction_data[scale_cols])
pdf_scaled = pd.DataFrame(pdf_scaled)
pdf_scaled.columns = scale_cols

pdf_scaled

In [ ]:
pdf_scaled.columns

In [ ]:
from sklearn.model_selection import train_test_split

prediction_feature=pdf_scaled[['평균기온(℃)', '평균최고기온(℃)', '최고기온(℃)', '평균최저기온(℃)', '최저기온(℃)', '강수량', '인구','아황산가스', '일산화탄소', '이산화질소']]
prediction_feature

In [ ]:
rdr_prediction=model_rfr.predict(prediction_feature)

In [ ]:
predict_evaluation = pd.DataFrame(rdr_prediction, columns = ['prediction'])
predict_evaluation

In [ ]:
pdf_scaled

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_rfr = RandomForestRegressor()
model_rfr.fit(x_train, y_train)
rfr_pred_y = model_rfr.predict(test_feature)

plt.plot(test_label.array, label = 'actual')
plt.plot(rfr_pred_y, label = 'prediction')
plt.legend()
plt.show()

# **model validation evaluation**

In [ ]:
from sklearn.metrics import mean_squared_error
import math
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

In [ ]:
# train rmse
train_predict = model_rfr.predict(x_train)
print("RMSE':{}".format(math.sqrt(mean_squared_error(train_predict, y_train))) )
 
# validation rmse
valid_predict = model_rfr.predict(x_valid)
print("RMSE':{}".format(math.sqrt(mean_squared_error(valid_predict, y_valid))) )
 
